PC: The later code blocks in this notebook are broken due to the new package structure. 

In [2]:
using StatsBase
using Combinatorics

using Pkg; Pkg.activate(".")
using HypergraphModularity

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [3]:
## Generate a graph
n = 200
Z = rand(1:3, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = partitionIntensityFunction(ω, kmax)

H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [7, 10, 9, 5, 2, 4, 4, 6, 7, 6  …  11, 5, 8, 7, 14, 12, 10, 5, 7, 8]


In [4]:
## Or, load one of the real world datasets

kmax = 4
H,labels = read_hypergraph_data("congress-bills",kmax)
H,labels = read_hypergraph_data("contact-primary-school-classes",kmax)
n = length(labels)

242

In [5]:
## Warm start using Clique-expansion Louvain

c_warm = CliqueExpansionModularity(H)
Zsing = collect(1:n)
Ω = estimateΩEmpirically(H, c_warm)
mwarm = modularity(H,c_warm,Ω,α=α0)

-214929.2375688453280698179324833138363700801833289531447208500845435369397407044

In [6]:
## Run Louvain Algorithms

@time Zlou = HyperLouvain(H,kmax,Ω;α=α0)
@time Zlou0 = HyperLouvain_0(H,kmax,Ω;α=α0) # uses compute_moddiff
@time Znew, changed = SuperNodeStep(H,Zsing,kmax,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou0 = modularity(H,Zlou0,Ω,α=α0)
msu = modularity(H,Znew,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
  6.702055 seconds (47.91 M allocations: 2.473 GiB, 8.71% gc time)


UndefVarError: UndefVarError: HyperLouvain_0 not defined

In [ ]:
## Try earlier version

@time Zlou = HyperLouvain(H,kmax,Ω;α=α0)
@time Znew, changed = SuperNodeStep(H,Zsing,kmax,Ω;α=α0)
msup = modularity(H,Znew,Ω,α=α0)
mlou = modularity(H,Zlou,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
 13.049563 seconds (109.09 M allocations: 10.466 GiB, 12.82% gc time)

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5